# DRL Approach for Optimal Precoder Design in MU-MISO: High SNR & Equal Power Constraint

## Overview
This project explores a Deep Reinforcement Learning (DRL) approach for designing optimal precoder vectors in a Multi-User MISO (MU-MISO) system, specifically under **High Signal-to-Noise Ratio (SNR)** conditions with a constraint of equal power allocation for each user. The aim is to maximize the sum rate, addressing high-SNR scenarios where signal quality is generally strong. The constraint that each user has unit power further challenges the decision-making process, requiring precise tuning for optimal performance.

## General High SNR
This version focuses on high-SNR conditions, where the signal quality is robust and noise has a minimal impact. Training under these conditions allows the DRL agent to leverage high-quality channels to achieve efficient precoder design within the equal power constraint.

### Objectives
- **Maximized Efficiency in High SNR:** Enable the model to capitalize on high-SNR conditions for efficient precoding strategies that optimize sum rate.
- **Equal Power Allocation Constraint:** Enforce a strict equal power allocation constraint, ensuring that each user’s power remains at a unit level, thereby adding an extra layer of complexity.
- **Comparison with Traditional Methods:** Benchmark the DRL-based approach against traditional linear precoding techniques such as MRT, ZF, and MMSE to highlight the advantages of DRL in constrained high-SNR environments.

### Implementation Details
- **Input Configuration:** The model architecture integrates SNR and channel information, enhancing the model’s ability to dynamically adapt in high-SNR environments.
- **Equal Power Constraint:** The DRL agent is trained to maintain equal power allocation across users, optimizing for sum rate within these bounds.
- **Training Process:** Training is conducted in high-SNR environments to allow the agent to develop robust strategies, focusing on performance maximization within the constraint.
- **Results Analysis:** Compare the DRL-based model’s outcomes with traditional techniques, evaluating the sum rate and overall efficiency of the system in high-SNR conditions.

In [1]:
# import Packeges
import numpy as np
import tensorflow as tf
import os

from DDPG_MultiUser import Agent
from utils import plot_learning_curve

In [2]:
# We use this to build the first run based on the results of the previous run.
changing_alg = 'logReward_constrain_highSNR'
prev_run_num ='3'
run_num = '4'
figure_file = 'plots/model_'+changing_alg+'/'

In [3]:
snr_user = 10   ##   Standard deviation of channel noice
n_tx = 6   # number of atennas
n_users = 4 # number of users
# This is the importance of each user  
weight_rate = np.full((n_users,),0.25)

# Helpful functions

In [4]:
def turn_mat_to_complex(matrix,n_tx = 6):
    """Turn real matrix to complex matrix."""
    if matrix.shape[0] ==1:
        matrix = np.reshape(matrix,(-1,2*n_tx))
    return matrix[:,:n_tx]+ 1j*matrix[:,n_tx:]

def PAE(states,n_tx = 6):
    """Phase ambiguity elimination to reach a unique solution."""
    h_ = turn_mat_to_complex(states,n_tx)
    h_zero_phase = h_*np.reshape(np.exp(-1j*np.angle(h_))[:,0],(-1,1))
    h_pae_row = np.concatenate([np.real(h_zero_phase),np.imag(h_zero_phase)],axis=1).reshape((1,-1))
    return h_pae_row

## Channel Model

The channel model can be represented as:

$$
\mathbf{Y} = \sqrt{\rho} \cdot \mathbf{H} \mathbf{W}^H \mathbf{s} + \mathbf{n}
$$

where:

- ρ is the input SNR, defined as ρ = Pt / σn².
- H is a matrix with dimensions K x n_tx.

### Definitions:

- K: Number of users.
- n<sub>tx</sub>: Number of antennas at the base station (BS).

In [5]:
def channel_states(n_users = 4, n_tx = 6, seed_number = 42 , random = False):
    """Channel model."""
    # H represents the channel states and has a size of (# of users, # of transmitted antennas).
    if not random:
        np.random.seed(seed_number)
    real =(1/np.sqrt(2))  * np.random.randn(n_users,n_tx).astype(np.float32)  # 4x 6
    imag = (1/np.sqrt(2)) * np.random.randn(n_users,n_tx).astype(np.float32) # 4x6
    H = np.hstack([real , imag]) #4x 12
    H = np.reshape(H,(1,-1)) #1 x 12*4(48)
    return H

# The Reward Equation

### The reward equation is given by:

$$
\text{Reward} = \sum_{k=1}^{K} (\nu_k R_k) = \sum_{k=1}^{K} \left( \nu_k \cdot \log_2 \left( 1 + \frac{(\rho |h_k \cdot w_k^H|)^2}{1 + \rho \sum_{j=1, j \neq k}^{K} |h_k \cdot w_j^H|^2} \right) \right)
$$

### Where:
- ρ is the input SNR
- K: Number of users.
- n<sub>tx</sub>: Number of antennas at the base station (BS).
- h: Channel state.
- w: Precoder vector.

In [6]:

def get_reward_vectorize(h_estimate, precoder_mat,n_tx = 6,snr = 1.0):
    """Get reward as a vector."""
    # Reward equation sum weighted rate of users.
    h_estimate = turn_mat_to_complex(h_estimate,n_tx) # (# of users, # of transmitted antennas)
    
    precoder_mat = turn_mat_to_complex(precoder_mat,n_tx) # (# of users, # of transmitted antennas)
    precoder_mat_H = (precoder_mat).conjugate().T  # (# of transmitted antennas, # of users) due to .T
    ## Multiplication of the channel estimate matrix by the precoder vector.
    #   - The diagonal elements represent the power without interference, 
    #     so hi_py_wi[i, i] is the power at user i.
    #   - The off-diagonal elements represent the interference, 
    #     so hi_py_wi[i, j] where i ≠ j represents the interference at user i.
    hi_py_wi = snr*np.square(np.abs(np.matmul(h_estimate,precoder_mat_H))) 
    power = np.diag(hi_py_wi) 
    interference = hi_py_wi.copy()
    # Make diognal zero so sum of each Row is the interference only
    np.fill_diagonal(interference,0) 

    interference = 1 +np.sum(interference,axis=1)
    reward_vector =  np.log2(1+power/interference)
    return reward_vector

# Function for Evaluation
This function is highly optimized with vectorization, allowing it to run very quickly.


In [7]:
def turn_3d_to_C(csi,n_tx = 6):
    """Convert a 3D real array into a complex array."""
    # CSI stands for Channel State Information.
    #   csi.shape[0] is the number of instances I want to evaluate.
    matrix = np.reshape(csi,(csi.shape[0],-1,2*n_tx))
    return matrix[:,:,:n_tx]+ 1j*matrix[:,:,n_tx:]

def PAE_3d(H,n_tx = 6):
    """Perform phase ambiguity elimination for a 3D matrix."""
    H_c = turn_3d_to_C(H,n_tx)
    H_zero_phase = H_c*np.exp(-1j*np.expand_dims(np.angle(H_c)[:,:,0],axis=2))
    h_pae_row = np.concatenate([np.real(H_zero_phase),np.imag(H_zero_phase)],axis=2)
    return h_pae_row

## Generate H Matrix

3-D matrix dimensions: (number of instances, number of users, number of transmitted antennas)


In [8]:
def channel_mat_seed(n_runs =5000,n_users = 4, n_tx = 6, lower_seed = 0):
    """Generate an H matrix where each row is initialized with a different seed."""
    matrix = np.zeros((n_runs,n_users,n_tx*2),dtype=np.float32)
    for i in range(lower_seed,n_runs + lower_seed):
        np.random.seed(i)
        real =(1/np.sqrt(2))  * np.random.randn(n_users,n_tx).astype(np.float32)
        imag = (1/np.sqrt(2)) * np.random.randn(n_users,n_tx).astype(np.float32)
        matrix[i-lower_seed] = np.hstack([real , imag])
    return matrix

def channel_matrix(n_evaluation ,n_users = 4,n_tx= 6, seed_number = 42 ,random = False):
    """Generate an H matrix where each row is fully random and initialized with a different seed."""
    if not random:
        np.random.seed(seed_number)
    real =(1/np.sqrt(2))  * np.random.randn(n_evaluation,n_users,n_tx).astype(np.float32)
    imag = (1/np.sqrt(2)) * np.random.randn(n_evaluation,n_users,n_tx).astype(np.float32)
    H = np.dstack([real , imag])
    # H = np.reshape(H,(n_evaluation,1,-1))
    return H

## Function to Calculate Reward for 3D H Matrix

This function calculates the reward for a 3D channel states matrix.

**Parameters:**

- **H**: A 3D channel states matrix with dimensions (number of instances, number of users, number of transmitted antennas).
- **W**: A precoder matrix with the same dimensions as H.

**Returns:**

- A reward matrix with dimensions (number of instances, number of users) for all instances and users.

In [9]:
def reward_for_3d(H_3d, precoder_3d,snr = 1.0):
    """Compute the reward function for multiple instances using vectorization to speed up computations."""
    n_tx = H_3d.shape[2]//2   ## Number of transmitted antennas
    
    ### Turn 3d matrix to Comlex
    h_estimate = turn_3d_to_C(H_3d,n_tx) # Has size (n_runs x n_user x n_tx)
    precoder_3d = turn_3d_to_C(precoder_3d,n_tx)
    # Transpose along the y and z axes.
    precoder_3d_t = np.transpose(precoder_3d.conjugate(),axes=[0 ,2 ,1]) # Has size (n_runs x n_tx x n_user)
    ### Get all combination of   |hi*wi|**2
    H_py_W = snr*np.square(np.abs(np.matmul(h_estimate,precoder_3d_t))) # Has size (n_runs x n_user x n_user)
    
    ### Diagonal elements
    power = np.diagonal(H_py_W,axis1=1,axis2=2)  # Has size (n_runs x n_user )
    ###  Fill diagonal with zeros
    interference = H_py_W.copy()
    diagonal_index = np.arange(H_3d.shape[1])
    interference[:, diagonal_index, diagonal_index] = 0
    ### Get the off diagonal elments
    interference = 1.0 +np.sum(interference,axis=2)
    reward_vector =  np.log2(1+power/interference)
    return reward_vector

## Linear Precoder Techniques to Evaluate Our Model

1. **MRT**  (Maximum Ratio Transmission)
2. **ZF**   (Zero-Forcing)
3. **MMSE** (Minimum Mean Square Error)

The precoder matrices are defined as follows:

$$
\begin{align*}
W_{MRT} & = H \\
W_{ZF} & = H(H^H H)^{-1} \\
W_{MMSE} & = H(H^H H + \frac{1}{\rho} I_K)
\end{align*}
$$

In [10]:
def sum_norm_root_square(H):
    """Calculate the summation of the squared norm of the channel matrix H."""
    sum_norm = np.sqrt(np.sum(np.square(np.linalg.norm(H,axis=2)),axis=1))
    sum_norm = np.repeat(sum_norm,H.shape[1]).reshape(H.shape[0],H.shape[1])  ### To help us on Broadcast
    sum_norm = np.expand_dims(sum_norm,axis=2)       ### has n_runs x n_users x1 dimentions
    return sum_norm

def MRT(H):  ### n_runs x n_user x n_tx*2
    """MRT stands for Maximum Ratio Transmission, a linear precoder technique."""
    precoder = H/np.expand_dims(np.linalg.norm(H,axis=2),axis=2)
    return precoder

def zf_mat(H):
    """ZF stands for Zero-Forcing, a linear precoder technique."""
    n_tx = H.shape[2]//2
    H = turn_3d_to_C(H,n_tx=n_tx)
    zf_H = np.matmul(np.linalg.inv(np.matmul(H,np.transpose(H.conjugate(),axes=[0,2,1]))),H)
    precoder_c  = zf_H/np.expand_dims(np.linalg.norm(zf_H,axis=2),axis=2)
    precoder = np.concatenate([np.real(precoder_c),np.imag(precoder_c)],axis=2)
    return precoder

def mmse_mat(H,snr= 1.0):
    """MMSE stands for Minimum Mean Square Error, a linear precoder technique."""
    n_tx = H.shape[2]//2
    H = turn_3d_to_C(H,n_tx=n_tx)
    identity_3d = np.zeros((H.shape[0],H.shape[1],H.shape[1]),dtype=np.float32)
    idx = np.arange(H.shape[1])
    identity_3d[:,idx,idx] = 1/snr
    mmse_H = np.matmul(np.linalg.inv(np.matmul(H,np.transpose(H.conjugate(),axes=[0,2,1]))+identity_3d),H)
    precoder_c  = mmse_H/np.expand_dims(np.linalg.norm(mmse_H,axis=2),axis=2)
    precoder = np.concatenate([np.real(precoder_c),np.imag(precoder_c)],axis=2)
    return precoder

### Put all together.

In [11]:
def evaluate_while_training(state_for_eval, agent, snr_user):
    """Evaluate the performance of the agent during training compared to other methods."""
    # Calculate cumulative rewards for the Linear techniques
    cumulative_reward_mrt = reward_for_3d(state_for_eval, MRT(state_for_eval), snr=snr_user)
    cumulative_reward_zf = reward_for_3d(state_for_eval, zf_mat(state_for_eval), snr=snr_user)
    cumulative_reward_mmse = reward_for_3d(state_for_eval, mmse_mat(state_for_eval, snr=snr_user), snr=snr_user)
    
    # Get the action matrix from the agent based on the evaluated state, reshaped for the input format
    action_mat_in = tf.reshape(
        agent.choose_action(state_for_eval.reshape((state_for_eval.shape[0], 1, -1)), evaluate=True),
        (state_for_eval.shape[0], n_users, -1))
    
    # Calculate rewards for the actions chosen by the agent using the channel state
    reward_mat_in = reward_for_3d(state_for_eval, action_mat_in, snr=snr_user)
    
    # Average the rewards obtained by the actor (agent) across all users
    ave_reward_actor = np.mean(reward_mat_in, axis=0)
    
    # Average the cumulative rewards for the linear methods
    ave_reward_mat_mrt = np.mean(cumulative_reward_mrt, axis=0)
    ave_reward_mat_zf = np.mean(cumulative_reward_zf, axis=0)
    ave_reward_mat_mmse = np.mean(cumulative_reward_mmse, axis=0)
    
    # Calculate the percentage of the average reward from the actor compared to the Linear methods
    percentage_mrt = np.sum(np.mean(reward_mat_in, axis=0)) * 100.0 / np.sum(ave_reward_mat_mrt)
    percentage_zf = np.sum(np.mean(reward_mat_in, axis=0)) * 100.0 / np.sum(ave_reward_mat_zf)
    percentage_mmse = np.sum(np.mean(reward_mat_in, axis=0)) * 100.0 / np.sum(ave_reward_mat_mmse)
    # Return the average rewards and percentages for MRT, ZF, and MMSE comparisons
    return ave_reward_actor, ave_reward_mat_mrt, ave_reward_mat_zf, ave_reward_mat_mmse, percentage_mrt, percentage_zf, percentage_mmse

# Agent Initialization

This code initializes an instance of the `Agent` class with the following parameters:

- **input_dims**: A tuple representing the dimensions of the input, calculated as (2 x number of antennas x number of users)
- **alpha**: Learning rate, set to 3.3e-5.
- **beta**: Weight decay parameter, set to 6.7e-5.
- **gamma**: Discount factor, set to 0.1.
- **n_users**: The number of users.
- **n_tx**: The number of transmitted antennas.
- **max_size**: The maximum size of the replay buffer, set to 1,000,000.
- **tau**: Soft update parameter, set to 0.005.
- **fc1**, **fc2**, **fc3**: Number of neurons in the first, second, and third fully connected layers, set to 1024, 512, and 128, respectively.
- **batch_size**: Size of the mini-batch for training, set to 64.
- **noise**: Noise of the channel, set to 3.0e-4.
- **changing_alg**: The name of the path to save the model.

The agent will use these parameters to interact with the environment and learn an optimal policy.


In [12]:
agent = Agent(
    input_dims=(2 * n_tx * n_users,),
    alpha=2e-05,
    beta=4e-05,
    gamma=0.05,
    n_users=n_users,
    n_tx=n_tx,
    max_size=1000000,
    tau=0.005,
    fc1=1024,
    fc2=512,
    fc3=128,
    batch_size=64,
    noise=2.0e-4,
    changing_alg=changing_alg
)

In [13]:
alpha = agent.alpha 
beta = agent.beta
noise = agent.noise

### Loading Previous Weights

This section is responsible for loading the previous weights, allowing the agent to resume learning from the results of the previous run. This functionality can help improve the convergence of the learning process by leveraging previously acquired knowledge. -->

In [14]:
### uncomment to be suitble in kaggle
# path = agent.model_path
# agent.model_path = "../input/last-lsnr/"+agent.model_path
if(os.path.exists(agent.model_path)):
    agent.learn(evaluate=True)
    agent.load_models(name='At_end_'+prev_run_num)
# agent.model_path = path

____Loading model of (logReward_constrain_highSNR) version At_end_3____


# Training the agent

In [33]:
n_episode = 5
n_time_steps = 100 

In [35]:
evaluate = False
agent.check_path('plots/model_' + changing_alg)

num_of_print = 1
best_ave = -np.inf
score_history = []

# Initialize matrices for evaluations
states_mat_3d = np.zeros((n_episode, n_time_steps, n_users, n_tx * 2), dtype=np.float32)

reward_mat = np.zeros((n_episode, num_of_print, n_users), dtype=np.float32)
ave_reward_mat_mrt = np.zeros((n_episode, num_of_print, n_users), dtype=np.float32)
ave_reward_mat_zf = np.zeros((n_episode, num_of_print, n_users), dtype=np.float32)
ave_reward_mat_mmse = np.zeros((n_episode,num_of_print,n_users),dtype=np.float32)
precoder_MRT = np.zeros((n_episode, num_of_print), dtype=np.float32)
precoder_zf = np.zeros((n_episode, num_of_print), dtype=np.float32)
precoder_mmse= np.zeros((n_episode,num_of_print),dtype=np.float32)

# Start the episodes
for episode in range(n_episode):
    score = 0
    count = 0
    
    # Initialize channel state
    states = PAE(channel_states(n_users, n_tx, seed_number=0), n_tx)
    
    for time_step in range(n_time_steps):        
        # Choose action based on current state
        actions = agent.choose_action(states, evaluate)
        
        # Calculate reward
        reward_vector = get_reward_vectorize(states, actions, n_tx=n_tx, snr=snr_user)
        reward = np.dot(reward_vector, weight_rate)

        # Store the current state
        states_3d = np.expand_dims(states.reshape((n_users, n_tx * 2)), axis=0)
        states_mat_3d[episode, time_step] = states_3d[0]

        # Update SNR for the next time step
        next_states = PAE(channel_states(n_users, n_tx, seed_number=time_step, random=True), n_tx)

        # Cumulative reward calculation
        score += reward  
        agent.remember(states, actions, reward, next_states)
        
        # Update states for the next iteration
        states = next_states
        
        if not evaluate:  # mean evaluate = False
            agent.learn()

        # Evaluation section
        # We choose not to evaluate at every step, so the evaluation frequency is controlled by the num_of_print variable.
        if (time_step + 1) % (n_time_steps // num_of_print) == 0:
            state_for_eval = states_mat_3d[episode, 0:(time_step + 1)]
            ave_ac,ave_mrt,ave_zf,ave_mmse,p_mrt,p_zf,p_mmse = evaluate_while_training(state_for_eval, agent, snr_user)

            reward_mat[episode, count] = ave_ac
            ave_reward_mat_mrt[episode, count] = ave_mrt
            ave_reward_mat_zf[episode, count] = ave_zf
            ave_reward_mat_mmse[episode,count] = ave_mmse

            precoder_MRT[episode, count] = p_mrt
            precoder_zf[episode, count] = p_zf
            precoder_mmse[episode,count] = p_mmse
            
            # Print evaluation results
            print(f"The percentage MRT reward = {p_mrt}")
            print(f"The percentage ZF reward = {p_zf}")
            print(f"The percentage mmse reward = {p_mmse}")
            agent.save_models(name='safe_' + run_num)
            count += 1
            print("\n")

    score_history.append(score)

    # Save matrices for analysis
    np.save(f'{figure_file}reward_Matrix_over_{n_episode}_{n_time_steps}_time_step_{changing_alg}_run_{run_num}.npy', reward_mat)
    np.save(f'{figure_file}States_Matrix_over_{n_episode}_{n_time_steps}_time_step_{changing_alg}_run_{run_num}.npy', states_mat_3d)
    np.save(f'{figure_file}reward_MRT_Matrix_over_{n_episode}_{n_time_steps}_time_step_{changing_alg}_run_{run_num}.npy', ave_reward_mat_mrt)
    np.save(f'{figure_file}reward_ZF_Matrix_over_{n_episode}_{n_time_steps}_time_step_{changing_alg}_run_{run_num}.npy', ave_reward_mat_zf)
    np.save(f'{figure_file}percentage_mrt_{n_episode}_{n_time_steps}_time_step_{changing_alg}_run_{run_num}.npy', precoder_MRT)
    np.save(f'{figure_file}percentage_zf_{n_episode}_{n_time_steps}_time_step_{changing_alg}_run_{run_num}.npy', precoder_zf)

    # Adjusting the agent's parameters to promote convergence
    # - Decaying the noise level, learning rate (alpha), and weight decay parameter (beta)
    #   by dividing them by the current iteration index plus one. 
    #   This gradual reduction helps stabilize the training process and leads to better 
    #   convergence towards the optimal solution.
    agent.noise = noise / (episode + 1)
    agent.alpha = alpha / (episode + 1)
    agent.beta = beta / (episode + 1)

    if min(precoder_MRT[episode, -1], precoder_zf[episode, -1]) > best_ave:
        best_ave = min(precoder_MRT[episode, -1], precoder_zf[episode, -1])
        agent.save_models(name='best_' + run_num)

    print('episode', episode, 'score MRT %.1f' % precoder_MRT[episode, -1], 'score ZF %.1f' % precoder_zf[episode, -1], 'avg percentage %.1f' % best_ave)

# Save final model
agent.save_models(name='At_end_' + run_num)


The percentage MRT reward = 90.52336983016748
The percentage ZF reward = 169.64402250276402
The percentage mmse reward = 90.3287748818272
____Saving model of (logReward_constrain_lowSNR) version safe_3____


____Saving model of (logReward_constrain_lowSNR) version best_3____
episode 0 score MRT 90.5 score ZF 169.6 avg percentage 90.5
The percentage MRT reward = 90.84690414625335
The percentage ZF reward = 167.82177977774168
The percentage mmse reward = 90.64468611068595
____Saving model of (logReward_constrain_lowSNR) version safe_3____


____Saving model of (logReward_constrain_lowSNR) version best_3____
episode 1 score MRT 90.8 score ZF 167.8 avg percentage 90.8
The percentage MRT reward = 90.88587493442212
The percentage ZF reward = 171.9035887771707
The percentage mmse reward = 90.69972771416484
____Saving model of (logReward_constrain_lowSNR) version safe_3____


____Saving model of (logReward_constrain_lowSNR) version best_3____
episode 2 score MRT 90.9 score ZF 171.9 avg percent

In [36]:
print("The value of Alpha at end = ",agent.alpha)
print("The value of  Beta at end = ",agent.beta)
print("The value of noise at end = ",agent.noise)

The value of Alpha at end =  4.000000000000001e-06
The value of  Beta at end =  8.000000000000001e-06
The value of noise at end =  4e-05


# Load the agent

### If in folder

In [188]:
agent.load_models(name='best_'+run_num)

____Loading model of (general_snr) version best_2____


# Evaluate the agent

## Graphs

In [37]:
# Set evaluate flag to True to skip plotting
evaluate = True

if not evaluate:
    # Title and filename for the first plot
    title1 = "Percentage"
    figure1_name = f'Percentage_{run_num}.png'
    
    # Plot the learning curve for MRT and ZF percentages
    plot_learning_curve(
        y_=[precoder_MRT[:, -1], precoder_zf[:, -1]],
        lim=n_episode,
        label=["Percentage MRT", "Percentage ZF"],
        figure_file=figure_file + figure1_name,
        title=title1
    )

    # Calculate the average rewards for MRT, ZF, and the model
    average_mrt = np.matmul(ave_reward_mat_mrt[:, -1], weight_rate)
    average_zf = np.matmul(ave_reward_mat_zf[:, -1], weight_rate)
    average_model = np.matmul(reward_mat[:, -1], weight_rate)
    
    # Title and filename for the second plot
    title2 = "Average sum reward over episodes"
    figure2_name = f'Average sum reward_{run_num}.png'
    
    # Plot the learning curve for average rewards
    plot_learning_curve(
        y_=[average_model, average_mrt, average_zf],
        lim=n_episode,
        label=["Actor", "MRT", "ZF"],
        figure_file=figure_file + figure2_name,
        title=title2
    )
    
    # Title and filename for the third plot
    figure3_name = f'AverageOver_100_runNumIs_{run_num}.png'
    
    # Plot the learning curve for the score history
    plot_learning_curve(
        y_=[score_history],
        lim=n_episode,
        label=["a"],
        figure_file=figure_file + figure3_name
    )

## Numerical Evaluation

Calculate the reward for a large number of instances using the vectorized function and take the average.

1. **Generate or Define Instances**: Create a large dataset of instances (channel states) for which the reward needs to be calculated.
2. **Calculate the Precoder Matrix Using Actor**: Use the large dataset of instances to calculate the precoder matrix.
3. **Calculate Rewards**: Use the vectorized reward function to compute rewards for all instances at once.
4. **Average the Rewards**: Calculate the average reward across all instances.

### High SNR

In [18]:
n_time_steps_eval = 1000

# Generate a channel matrix without the PAE effect
H_without_pae = channel_matrix(n_time_steps_eval, n_tx=n_tx, random=True)
# H_without_pae = channel_matrix(n_time_steps_eval, n_tx=n_tx, seed_number=400)

# Apply the PAE effect to the channel matrix
H = PAE_3d(H_without_pae)

# Compute the precoders for MRT, ZF, and MMSE
W_mrt = MRT(H)
W_zf = zf_mat(H)
W_mmse = mmse_mat(H, snr=snr_user)

# Calculate the rewards for each precoder
reward_mrt = reward_for_3d(H, W_mrt, snr=snr_user)
reward_zf = reward_for_3d(H, W_zf, snr=snr_user)
reward_mmse = reward_for_3d(H, W_mmse, snr=snr_user)

# Choose action using the agent based on the current state and evaluate
action_mat = agent.choose_action(H.reshape((H.shape[0],1,-1)), evaluate=True)
action_mat = tf.reshape(action_mat, (n_time_steps_eval, n_users, -1))

# Calculate the evaluation rewards based on the chosen action
eva_reward_mat = reward_for_3d(H, tf.reshape(action_mat, (H.shape[0], n_users, -1)), snr=snr_user)

# Compute the mean rewards for MRT, ZF, MMSE, and the model's actions
rmrt = np.mean(reward_mrt, axis=0)
rzf = np.mean(reward_zf, axis=0)
rmmse = np.mean(reward_mmse, axis=0)
rmod = np.mean(eva_reward_mat, axis=0)

# Print the average rewards and performance metrics
print(f"Average reward by actor =\t {rmod}")
print(f"Average reward by MRT   =\t {rmrt}")
print(f"Average reward by ZF    =\t {rzf}")
print(f"Average reward by MMSE   =\t {rmmse}")
print(f"Percentage subject to MRT =\t {np.sum(rmod) * 100.0 / np.sum(rmrt)} %")
print(f"Percentage subject to ZF  =\t {np.sum(rmod) * 100.0 / np.sum(rzf)} %")
print(f"Percentage subject to MMSE =\t {np.sum(rmod) * 100.0 / np.sum(rmmse)} %")

Average reward by actor =	 [1.5799184 1.5734459 1.6192862 1.698712 ]
Average reward by MRT   =	 [1.7174187 1.7048019 1.6969966 1.6832829]
Average reward by ZF    =	 [4.7038627 4.71586   4.695405  4.708968 ]
Average reward by MMSE   =	 [4.767459  4.775095  4.7577763 4.7699056]
Percentage subject to MRT =	 95.13211140121284 %
Percentage subject to ZF  =	 34.378075208972355 %
Percentage subject to MMSE =	 33.93435741450343 %


### Single Instance (State) Evaluation

In [39]:
# Generate channel states using the PAE effect with a fixed seed for reproducibility
h = PAE(channel_states(n_tx=n_tx, seed_number=42, random=True))

# Reshape the channel states to 3D for processing
h_3d = np.expand_dims(h.reshape((n_users, n_tx * 2)), axis=0)

# Calculate rewards using different precoding techniques
reward_mrt_sample = reward_for_3d(h_3d, MRT(h_3d), snr=snr_user)[0]
reward_zf_sample = reward_for_3d(h_3d, zf_mat(h_3d), snr=snr_user)[0]
reward_mmse_sample = reward_for_3d(h_3d, mmse_mat(h_3d), snr=snr_user)[0]

# Choose action using the agent based on the current state and evaluate
precoder = agent.choose_action(h, evaluate=True)

# Calculate the reward for the chosen action
reward_model_sample = get_reward_vectorize(h, precoder, snr=snr_user)

# Print the rewards obtained from the actor and different methods
print(f"Reward by actor =\t {reward_model_sample}")
print(f"Reward by MRT   =\t {reward_mrt_sample}")
print(f"Reward by ZF    =\t {reward_zf_sample}")
print(f"Reward by MMSE  =\t {reward_mmse_sample}")

# Calculate and print the percentage performance relative to each method
print(f"Percentage subject to MRT  =\t {np.sum(reward_model_sample) * 100.0 / np.sum(reward_mrt_sample)} %")
print(f"Percentage subject to ZF   =\t {np.sum(reward_model_sample) * 100.0 / np.sum(reward_zf_sample)} %")
print(f"Percentage subject to MMSE  =\t {np.sum(reward_model_sample) * 100.0 / np.sum(reward_mmse_sample)} %")

Reward by actor =	 [0.10820762 0.05179625 0.12350858 0.06635842]
Reward by MRT   =	 [0.129981   0.05977865 0.12960957 0.08277049]
Reward by ZF    =	 [0.06295161 0.02585701 0.10064117 0.06044395]
Reward by MMSE  =	 [0.08865417 0.03249981 0.10822549 0.06965037]
Percentage subject to MRT  =	 87.0023194011903 %
Percentage subject to ZF   =	 140.00786161834372 %
Percentage subject to MMSE  =	 117.00199406484657 %
